In [1]:

# 単語分割

# 前処理

# 文章とラベルに分ける

# データを読み込む

# ボキャブラリーを作成

# DataLoaderの作成

#


# 単語分割

In [2]:
import MeCab
m_t = MeCab.Tagger('-Ochasen')

In [3]:
text = '機械学習が好きです'

In [4]:
print(m_t.parse(text).strip())

機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
EOS


In [5]:
def tokenizer_mecab(text):
    text = m_t.parse(text) # mecab
    ret = text.strip().split()  # text処理
    return ret

# 前処理

In [33]:
import re

def preprocessing_text(text):
    # 前処理
    text = re.sub('\r', '', text) # 
    text = re.sub('\n', '', text) # 改行
    text = re.sub(' ', '', text) # 半角
    text = re.sub('　　', '', text) # 全角
    text = re.sub(r'[0-9 ０-９]', '0', text)

    return text

# tokenizerの作成

In [34]:
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text) # textの前処理
    ret = tokenizer_mecab(text) # mecabの単語分割
    return ret

# torchtext

In [35]:
import torchtext

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                                                 use_vocab=True, lower=True, include_lengths=True, 
                                                 batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# Datasetの作成

In [36]:
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

In [39]:
vars(train_ds[0])

{'Text': ['王',
  'オウ',
  '王',
  '名詞-一般',
  'と',
  'ト',
  'と',
  '助詞-並立助詞',
  '王子',
  'オウジ',
  '王子',
  '名詞-一般',
  'と',
  'ト',
  'と',
  '助詞-並立助詞',
  '女王',
  'ジョオウ',
  '女王',
  '名詞-一般',
  'と',
  'ト',
  'と',
  '助詞-並立助詞',
  '姫',
  'ヒメ',
  '姫',
  '名詞-一般',
  'と',
  'ト',
  'と',
  '助詞-並立助詞',
  '男性',
  'ダンセイ',
  '男性',
  '名詞-一般',
  'と',
  'ト',
  'と',
  '助詞-並立助詞',
  '女性',
  'ジョセイ',
  '女性',
  '名詞-一般',
  'が',
  'ガ',
  'が',
  '助詞-格助詞-一般',
  'い',
  'イ',
  'いる',
  '動詞-自立',
  '一段',
  '連用形',
  'まし',
  'マシ',
  'ます',
  '助動詞',
  '特殊・マス',
  '連用形',
  'た',
  'タ',
  'た',
  '助動詞',
  '特殊・タ',
  '基本形',
  '。',
  '。',
  '。',
  '記号-句点',
  'eos'],
 'Label': '0'}

# ボキャブラリーの作成

In [42]:
TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.freqs

Counter({'王': 2,
         'オウ': 1,
         '名詞-一般': 17,
         'と': 10,
         'ト': 5,
         '助詞-並立助詞': 5,
         '王子': 2,
         'オウジ': 1,
         '女王': 2,
         'ジョオウ': 1,
         '姫': 2,
         'ヒメ': 1,
         '男性': 2,
         'ダンセイ': 1,
         '女性': 2,
         'ジョセイ': 1,
         'が': 6,
         'ガ': 3,
         '助詞-格助詞-一般': 10,
         'い': 1,
         'イ': 1,
         'いる': 5,
         '動詞-自立': 6,
         '一段': 3,
         '連用形': 6,
         'まし': 1,
         'マシ': 1,
         'ます': 5,
         '助動詞': 9,
         '特殊・マス': 3,
         'た': 2,
         'タ': 1,
         '特殊・タ': 1,
         '基本形': 8,
         '。': 12,
         '記号-句点': 4,
         'eos': 4,
         '機械': 2,
         'キカイ': 1,
         '学習': 2,
         'ガクシュウ': 1,
         '名詞-サ変接続': 8,
         '好き': 2,
         'スキ': 1,
         '名詞-形容動詞語幹': 4,
         'です': 2,
         'デス': 1,
         '特殊・デス': 1,
         '本章': 4,
         'ホンショウ': 2,
         'から': 2,
         'カラ': 1,
         '自然

# DataLoaderの作成

In [44]:
train_dl= torchtext.data.Iterator(train_ds, batch_size=2, train=True)
val_dl = torchtext.data.Iterator(val_ds, batch_size=2, train=False, sort=False)
test_dl = torchtext.data.Iterator(test_ds, batch_size=2, train=False, sort=False)

In [48]:
# 動作確認
batch = next(iter(train_dl))
print(batch.Text)
print(batch.Label)

(tensor([[ 33,  63,  33,   2,  50,  90,  50,   5,  83,  95,  83,  32,  84,  93,
          84,   2,  66,  97,  66,   9,  55, 109,  55,   5, 119],
        [ 33,  63,  33,   2,  53, 106,  53,   5,  57, 111,  57, 116,  71,  96,
          71,   2,  45,  45,  45,   9,   8,  28,   8,   5,  82]]), tensor([25, 25]))
tensor([1, 0])


# IMDbデータセットの実装

In [ ]:
import glob
